In [49]:
%%writefile serial_diffusion.cc
#include <stdio.h>
#include <math.h>
#include <fstream>
#include <cstdlib>
#include <cstdio>
#include <algorithm>
#include <vector>
#include <iostream>
#include <fstream>
#include <sstream>

bool isin(std::vector<float> v, float x)
{

if(std::find(v.begin(), v.end(), x) != v.end()) {
    /* v contains x */
	return true;
} else {
    /* v does not contain x */
	return false;
}

}

std::ostream& filewrite(std::ostream& os, double* arr1, int len1, double* arr2, int len2)
{
    char buf[5];
    for (int i = 0; i < len1; ++i)
    {
       sprintf(buf, "%.3f", arr1[i]);
       os << std::string(buf) <<" ";
    }

	os << std::endl;

    for (int j = 0; j < len2; ++j)
    {
       sprintf(buf, "%.3f", arr2[j]);
       os << std::string(buf) <<" ";
    }

    return os;
}



void mul_by_tridiag(double* res, double* arr, int len_arr, float lower_d, float d, float upper_d)
{
        res[0] = d * arr[0] + upper_d * arr[1];
        res[len_arr-1] = lower_d * arr[len_arr-2] + d * arr[len_arr-1];
        for (int i = 1; i < len_arr - 1; i++) {
            res[i] = lower_d * arr[i-1] + d * arr[i] + upper_d * arr[i+1];
	}
}


void inter_diffusion(double* B, double* alpha, double* beta, double* n, int ny, float s, float a)
{
	mul_by_tridiag(B, &n[1], ny - 2, 0.5 * s * (1 - a), 1 - s, 0.5 * s * (1 + a));
	B[0] = B[0] + 0.5 * s * (n[0] + n[0]) * (1 - a); // changed
	B[ny-3] = B[ny-3] + 0.5 * s * (n[ny-1] + n[ny-1]) * (1 + a); // changed

	alpha[0] = (s * (1 + a) / 2) / (1 + s);
	beta[0] = B[0] / (1 + s);

	for (int i = 0; i < ny - 3; i++){
		alpha[i] = (s * (1 + a) / 2) / (1 + s - s * (1 - a) / 2 * alpha[i - 1]);
		beta[i] = (s * (1 - a) / 2 * beta[i - 1] + B[i]) / (1 + s - s * (1 - a) / 2 * alpha[i - 1]);
	}

	n[ny - 2] = (B[ny - 3] + s * (1 - a) / 2 * beta[ny - 4]) / (1 + s - s * (1 - a) / 2 * alpha[ny - 4]);

	for (int j = ny - 3; j > 0; j--){
            n[j] = alpha[j - 1] * n[j + 1] + beta[j - 1];
	}
}



int main(int argc, char *argv[])
{
	if (argc < 5){
		printf("%s D_left a path_to_init_prof use_precomputed\n", argv[0]);
		return -1;
	}
	float y_max = 5.;
	float dy = 1e-3;
	float dt = 1e-6;
	float D_left = std::atof(argv[1]);
	float a = std::atof(argv[2]);
	float D_right = 2e-2;
	float s_left = D_left * dt / (dy * dy);
	float s_right = D_right * dt / (dy * dy);
	float init_prof_shift = 0;
	char filename[130]; // make sure it's big enough
	float curr_time;
	float arr[] = {1.};
	// float arr[] = {60};
    int n = sizeof(arr) / sizeof(arr[0]);
	std::vector<float> steps_to_dump(arr, arr + n);
	float sim_time = arr[n-1] + 2 * dt;
	int nt = (int) (sim_time / dt);

	for (int i=0; i < steps_to_dump.size(); i++){
		std::cout << steps_to_dump[i] << ' ';
	}
	std::cout << '\n';

	printf("D_left = %.3f, num of timesteps: %d, a = %.3f\n", D_left, nt, a);

	int ny_left = (int) (y_max / dy) + 1;
	int ny_right = (int) (2 * y_max / dy) + 2; // one for zero and one for type change
	double n_left[ny_left];
	double n_right[ny_right];

	bool use_precomputed;
	std::string action(argv[4]);
	if (action == "true") {
		use_precomputed = true;
	}
	else {
		use_precomputed = false;
	}


	if (use_precomputed){
		printf("Using precomputed profile\n");

		std::ifstream in(argv[3]);
		std::string line;
		float val;
		int ind1 = 0;
		int ind2 = 0;
		std::getline(in, line);
		std::stringstream ss1(line);
		while (ss1 >> val) {
			n_left[ind1] = val;
			ind1++;
		}

		std::getline(in, line);
		std::stringstream ss2(line);
		while (ss2 >> val) {
			n_right[ind2] = val;
			ind2++;
		}
	}
	else{
		printf("Using new initial profile\n");

		float p = -y_max;
		for (int j = 0; j < ny_left; j++){
			n_left[j] = (1 - tanhf((p - init_prof_shift) * 10)) / 2;
			// n_left[j] = (y_max - p) / y_max;
			p += dy;
		}

		p = - dy;
		for (int j = 0; j < ny_right; j++){
			n_right[j] = (1 - tanhf((p - init_prof_shift) * 10)) / 2;
			// n_right[j] = 0;
			p += dy;
		}

	}



    double *alpha_left;
	double *beta_left;
    double *B_left;
	alpha_left = new double[ny_left - 3];
	B_left = new double[ny_left - 2];
	beta_left = new double[ny_left - 3];

    double *alpha_right;
	double *beta_right;
    double *B_right;
	alpha_right = new double[ny_right - 3];
	B_right = new double[ny_right - 2];
	beta_right = new double[ny_right - 3];

	for (int j = 1; j < nt; j++){
		inter_diffusion(B_left, alpha_left, beta_left, n_left, ny_left, s_left, 0.);
		n_right[0] = (n_right[1] - D_left / D_right * (n_left[ny_left-1] - n_left[ny_left-2])) / (1 - 2 * a);
		n_left[ny_left-1] = n_right[1];

        inter_diffusion(B_right, alpha_right, beta_right, n_right, ny_right, s_right, a);
        n_left[ny_left-1] = n_left[ny_left-2] + D_right / D_left * (n_right[1] - n_right[0] * (1 - 2 * a));
		n_right[1] = n_left[ny_left-1];
		// n_right[ny_right-1] = (1 - 2 * a) * n_right[ny_right-2]; // zero outflux


		curr_time = j * dt;
		if (isin(steps_to_dump, curr_time)){
			sprintf(filename, "profiles/impulses_volt/5_10_sim_time_%.6f_D_left_%.3f_a_%.4f.txt", curr_time, D_left, a);
			std::fstream of(filename, std::ios::out);

			if (of.is_open()){
					filewrite(of, n_left, ny_left, n_right, ny_right);
					of.close();
			}
			printf("Dumped in a file a profile for %.2f seconds\n", curr_time);
			steps_to_dump.erase(std::remove(steps_to_dump.begin(), steps_to_dump.end(), curr_time), steps_to_dump.end());
		}
	}

	delete alpha_left;
	delete B_left;
	delete beta_left;
    delete alpha_right;
	delete beta_right;
    delete B_right;

	return 0;
}


Overwriting serial_diffusion.cc


In [50]:
%%writefile serial_experiment.sh
g++ serial_diffusion.cc -o run_serial_diffusion -lm -fopenmp

D_left=0.4
use_precomputed=true
init_prof_path=profiles/over_time/5_10_sim_time_40.00_D_left_0.400_a_-0.0017.txt
a=-0.0016525538567301913
time ./run_serial_diffusion $D_left $a $init_prof_path $use_precomputed

Writing serial_experiment.sh


In [51]:
!chmod +x serial_experiment.sh

!./serial_experiment.sh

serial_diffusion.cc: In function ‘std::ostream& filewrite(std::ostream&, double*, int, double*, int)’:
serial_diffusion.cc:38:26: warning: ‘sprintf’ may write a terminating nul past the end of the destination []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
   38 |        sprintf(buf, "%.3f", arr2[j]);
      |                          ^
serial_diffusion.cc:38:15: note: ‘sprintf’ output between 4 and 315 bytes into a destination of size 5
   38 |        sprintf(buf, "%.3f", arr2[j]);
      |        ~~~~~~~^~~~~~~~~~~~~~~~~~~~~~
serial_diffusion.cc:30:26: warning: ‘sprintf’ may write a terminating nul past the end of the destination []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
   30 |        sprintf(buf, "%.3f", arr1[i]);
      |                          ^
serial_diffusion.cc:30:15: note: ‘sprintf’ output between 4 and 315 bytes into a destination of size 5
  

#with openMP

In [54]:
%%writefile parallel_diffusion.cc
#include <stdio.h>
#include <math.h>
#include <fstream>
#include <cstdlib>
#include <cstdio>
#include <algorithm>
#include <vector>
#include <iostream>
#include <fstream>
#include <sstream>
#include <omp.h>

bool isin(const std::vector<float>& v, float x) {
    return std::find(v.begin(), v.end(), x) != v.end();
}

std::ostream& filewrite(std::ostream& os, const double* arr1, int len1, const double* arr2, int len2) {
    char buf[5];
    for (int i = 0; i < len1; ++i) {
        sprintf(buf, "%.3f", arr1[i]);
        os << std::string(buf) << " ";
    }
    os << std::endl;
    for (int j = 0; j < len2; ++j) {
        sprintf(buf, "%.3f", arr2[j]);
        os << std::string(buf) << " ";
    }
    return os;
}

void mul_by_tridiag(double* res, const double* arr, int len_arr, float lower_d, float d, float upper_d) {
    int tid;
    res[0] = d * arr[0] + upper_d * arr[1];
    res[len_arr - 1] = lower_d * arr[len_arr - 2] + d * arr[len_arr - 1];

    #pragma omp parallel for schedule(static) shared(d, upper_d, lower_d, res, arr)
    for (int i = 1; i < len_arr - 1; i++) {
        //tid = omp_get_thread_num();
        res[i] = lower_d * arr[i - 1] + d * arr[i] + upper_d * arr[i + 1];
        //printf("tid = %d, res[%d] = %f\n", tid, i, res[i]);
    }
}



void inter_diffusion(double* B, double* alpha, double* beta, double* n, int ny, float s, float a) {
    mul_by_tridiag(B, &n[1], ny - 2, 0.5 * s * (1 - a), 1 - s, 0.5 * s * (1 + a));
    B[0] = B[0] + 0.5 * s * (n[0] + n[0]) * (1 - a);
    B[ny - 3] = B[ny - 3] + 0.5 * s * (n[ny - 1] + n[ny - 1]) * (1 + a);

    alpha[0] = (s * (1 + a) / 2) / (1 + s);
    beta[0] = B[0] / (1 + s);

    for (int i = 1; i < ny - 3; i++) {
        alpha[i] = (s * (1 + a) / 2) / (1 + s - s * (1 - a) / 2 * alpha[i - 1]);
        beta[i] = (s * (1 - a) / 2 * beta[i - 1] + B[i]) / (1 + s - s * (1 - a) / 2 * alpha[i - 1]);
    }

    n[ny - 2] = (B[ny - 3] + s * (1 - a) / 2 * beta[ny - 4]) / (1 + s - s * (1 - a) / 2 * alpha[ny - 4]);

    #pragma omp parallel for schedule(static) shared(alpha, beta)
    for (int j = ny - 3; j > 0; j--) {
        n[j] = alpha[j - 1] * n[j + 1] + beta[j - 1];
    }
}

int main(int argc, char* argv[]) {
    if (argc < 5) {
        printf("%s D_left a path_to_init_prof use_precomputed\n", argv[0]);
        return -1;
    }

    float y_max = 5.;
    float dy = 1e-3;
    float dt = 1e-6;
    float D_left = std::atof(argv[1]);
    float a = std::atof(argv[2]);
    float D_right = 2e-2;
    float s_left = D_left * dt / (dy * dy);
    float s_right = D_right * dt / (dy * dy);
    float init_prof_shift = 0;
    char filename[130];
    float curr_time;
    float arr[] = { 1. };
    int n = sizeof(arr) / sizeof(arr[0]);
    std::vector<float> steps_to_dump(arr, arr + n);
    float sim_time = arr[n - 1] + 2 * dt;
    int nt = (int)(sim_time / dt);

    for (int i = 0; i < steps_to_dump.size(); i++) {
        std::cout << steps_to_dump[i] << ' ';
    }
    std::cout << '\n';

    printf("D_left = %.3f, num of timesteps: %d, a = %.3f\n", D_left, nt, a);

    int ny_left = (int)(y_max / dy) + 1;
    int ny_right = (int)(2 * y_max / dy) + 2;
    double* n_left = new double[ny_left];
    double* n_right = new double[ny_right];

    bool use_precomputed = (std::string(argv[4]) == "true");

    if (use_precomputed) {
        printf("Using precomputed profile\n");

        std::ifstream in(argv[3]);
        std::string line;
        float val;
        int ind1 = 0;
        int ind2 = 0;
        std::getline(in, line);
        std::stringstream ss1(line);
        while (ss1 >> val) {
            n_left[ind1++] = val;
        }

        std::getline(in, line);
        std::stringstream ss2(line);
        while (ss2 >> val) {
            n_right[ind2++] = val;
        }
    }
    else {
        printf("Using new initial profile\n");

        float p = -y_max;
        //#pragma omp parallel for schedule(static)
        for (int j = 0; j < ny_left; j++) {
            n_left[j] = (1 - tanhf((p - init_prof_shift) * 10)) / 2;
            p += dy;
        }

        p = -dy;
        //#pragma omp parallel for schedule(static)
        for (int j = 0; j < ny_right; j++) {
            n_right[j] = (1 - tanhf((p - init_prof_shift) * 10)) / 2;
            p += dy;
        }
    }

    double* alpha_left = new double[ny_left - 3];
    double* beta_left = new double[ny_left - 3];
    double* B_left = new double[ny_left - 2];

    double* alpha_right = new double[ny_right - 3];
    double* beta_right = new double[ny_right - 3];
    double* B_right = new double[ny_right - 2];

    for (int j = 1; j < nt; j++) {
        inter_diffusion(B_left, alpha_left, beta_left, n_left, ny_left, s_left, 0.);
        n_right[0] = (n_right[1] - D_left / D_right * (n_left[ny_left - 1] - n_left[ny_left - 2])) / (1 - 2 * a);
        n_left[ny_left - 1] = n_right[1];

        inter_diffusion(B_right, alpha_right, beta_right, n_right, ny_right, s_right, a);
        n_left[ny_left - 1] = n_left[ny_left - 2] + D_right / D_left * (n_right[1] - n_right[0] * (1 - 2 * a));
        n_right[1] = n_left[ny_left - 1];

        curr_time = j * dt;
        if (isin(steps_to_dump, curr_time)) {
            sprintf(filename, "profiles/impulses_volt/5_10_sim_time_%.6f_D_left_%.3f_a_%.4f.txt", curr_time, D_left, a);
            std::ofstream of(filename);

            if (of.is_open()) {
                filewrite(of, n_left, ny_left, n_right, ny_right);
                of.close();
            }
            printf("Dumped in a file a profile for %.2f seconds\n", curr_time);
            steps_to_dump.erase(std::remove(steps_to_dump.begin(), steps_to_dump.end(), curr_time), steps_to_dump.end());
        }
    }

    delete[] alpha_left;
    delete[] B_left;
    delete[] beta_left;
    delete[] alpha_right;
    delete[] beta_right;
    delete[] B_right;
    delete[] n_left;
    delete[] n_right;

    return 0;
}


Overwriting parallel_diffusion.cc


In [57]:
%%writefile parallel_experiment.sh
g++ parallel_diffusion.cc -o run_parallel_diffusion -lm -fopenmp

D_left=0.4
use_precomputed=true
init_prof_path=profiles/over_time/5_10_sim_time_40.00_D_left_0.400_a_-0.0017.txt
a=-0.0016525538567301913
time ./run_parallel_diffusion $D_left $a $init_prof_path $use_precomputed

Overwriting parallel_experiment.sh


In [58]:
!chmod +x parallel_experiment.sh

!./parallel_experiment.sh

parallel_diffusion.cc: In function ‘std::ostream& filewrite(std::ostream&, const double*, int, const double*, int)’:
parallel_diffusion.cc:25:27: warning: ‘sprintf’ may write a terminating nul past the end of the destination []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
   25 |         sprintf(buf, "%.3f", arr2[j]);
      |                           ^
parallel_diffusion.cc:25:16: note: ‘sprintf’ output between 4 and 315 bytes into a destination of size 5
   25 |         sprintf(buf, "%.3f", arr2[j]);
      |         ~~~~~~~^~~~~~~~~~~~~~~~~~~~~~
parallel_diffusion.cc:20:27: warning: ‘sprintf’ may write a terminating nul past the end of the destination []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
   20 |         sprintf(buf, "%.3f", arr1[i]);
      |                           ^
parallel_diffusion.cc:20:16: note: ‘sprintf’ output between 4 and 315 bytes int

#Plot the result

In [59]:
!chmod +x plot_img.sh

!./plot_img.sh 5

Skipping directory: profiles/impulses_volt/.ipynb_checkpoints
